In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize


# Пандемия_COVID-19
tables = pd.read_html(
    'https://ru.wikipedia.org/wiki/%D0%9F%D0%B0%D0%BD%D0%B4%D0%B5%D0%BC%D0%B8%D1%8F_COVID-19',
    match='стран')

In [177]:
len(tables)

3

In [178]:
df = tables[1]
df

Страны и территории  \
                                 Всего 000000000000000   
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN   
..                                                 ...   
251                                                NaN   
252                                                NaN   
253                                                NaN   
254                                         Примечания   
255  ↑ Всемирная организация здравоохранения опреде...   

                                                        \
                               Всего 000000000000000.1   
0                                        США[116][117]   
1                                        Бразилия[118]   
2                                           Индия[119]   
3                                          Россия[120]   
4                     Южно-Африканская Республика[121]   
..                                                 ...   
251                                Шарль де Голль[301]   
252                                 USS Kidd (DDG-993)   
253                                        Dolfijn[nl]   
254                                         Примечания   
255  ↑ Всемирная организация здравоохранения опреде...   

                                              Заражено  \
                                            23 700 085   
0                                            6 187 387   
1                                            3 608 895   
2                                            3 700 000   
3                                            1 000 048   
4                                              609 140   
..                                                 ...   
251                                               1081   
252                                                 78   
253                                                  8   
254                                         Примечания   
255  ↑ Всемирная организация здравоохранения опреде...   

                                               Выздор.  \
                                            16 100 057   
0                                            3 429 638   
1                                            2 384 302   
2                                            1 810 079   
3                                              729 411   
4                                              461 734   
..                                                 ...   
251                                                  0   
252                                                  0   
253                                                  —   
254                                         Примечания   
255  ↑ Всемирная организация здравоохранения опреде...   

                                                Умерло  \
                                               814 000   
0                                              187 156   
1                                              106 571   
2                                               49 171   
3                                               15 617   
4                                               11 556   
..                                                 ...   
251                                                  0   
252                                                  0   
253                                                  0   
254                                         Примечания   
255  ↑ Всемирная организация здравоохранения опреде...   

    .mw-parser-output .ts-comment-commentedText{border-bottom:1px dotted;cursor:help}@media(hover:none){.mw-parser-output .ts-comment-commentedText:not(.rt-commentedText){border-bottom:0;cursor:auto}}Летал.  \
                                                  

In [179]:
df.drop(('Страны и территории', 'Всего 000000000000000'), axis=1, inplace=True)
df.drop(axis=0, index=[254, 255], inplace=True)

In [180]:
df.columns = df.columns.droplevel(-1)

In [181]:
df.rename(columns={'.mw-parser-output .ts-comment-commentedText{border-bottom:1px dotted;cursor:help}@media(hover:none){.mw-parser-output .ts-comment-commentedText:not(.rt-commentedText){border-bottom:0;cursor:auto}}Летал.': 'Летал.'}, inplace=True)

In [182]:
df.replace({'\[[0-9]+\]': ''}, regex=True, inplace=True)

In [183]:
df.head()

,Страны и территории,Заражено,Выздор.,Умерло,Летал.,На 1 млн
0,США,6 187 387,3 429 638,187 156,29,16538
1,Бразилия,3 608 895,2 384 302,106 571,30,15412
2,Индия,3 700 000,1 810 079,49 171,16,1832
3,Россия,1 000 048,729 411,15 617,16,6289
4,Южно-Африканская Республика,609 140,461 734,11 556,19,9750


In [184]:
indices = df[df['Страны и территории'].str.contains('Справочно')].index

In [185]:
indices.size

4

In [186]:
indices = df[
    (df['Страны и территории'].str.contains('Справочно') |
     df['Страны и территории'].str.contains('Морские суда'))
].index
dfs = []
cur = 0
for idx in indices:
    if idx != indices[-1]:
        part = df.iloc[cur:idx, :]
    else:
        part = df[idx+1: ]
    cur = idx + 1
    dfs.append(part)

In [205]:
dfs[0]

,Страны и территории,Заражено,Выздор.,Умерло,Летал.,На 1 млн
0,США,6 187 387,3 429 638,187 156,29,16538
1,Бразилия,3 608 895,2 384 302,106 571,30,15412
2,Индия,3 700 000,1 810 079,49 171,16,1832
3,Россия,1 000 048,729 411,15 617,16,6289
4,Южно-Африканская Республика,609 140,461 734,11 556,19,9750
...,...,...,...,...,...,...
219,Самоа,н/д,NaN,NaN,NaN,NaN
220,Соломоновы Острова,н/д,NaN,NaN,NaN,NaN
221,Тонга,н/д,NaN,NaN,NaN,NaN
222,Тувалу,н/д,NaN,NaN,NaN,NaN


In [206]:
temp_df = dfs[0]
temp_df.columns = temp_df.columns.str.normalize('NFKC')
cols = temp_df.columns.drop(temp_df.columns[0])

In [213]:
for col in cols:
    temp_df.loc[:, col] = temp_df.loc[:, col].str.normalize('NFKC')
    temp_df.loc[:, col] = temp_df.loc[:, col].replace(
        {' ': '', 'н/д': np.nan}, regex=True
    ).astype('float32')

In [214]:
temp_df

,Страны и территории,Заражено,Выздор.,Умерло,Летал.,На 1 млн
0,США,6187387.0,3429638.0,187156.0,29.0,16538.0
1,Бразилия,3608895.0,2384302.0,106571.0,30.0,15412.0
2,Индия,3700000.0,1810079.0,49171.0,16.0,1832.0
3,Россия,1000048.0,729411.0,15617.0,16.0,6289.0
4,Южно-Африканская Республика,609140.0,461734.0,11556.0,19.0,9750.0
...,...,...,...,...,...,...
219,Самоа,NaN,NaN,NaN,NaN,NaN
220,Соломоновы Острова,NaN,NaN,NaN,NaN,NaN
221,Тонга,NaN,NaN,NaN,NaN,NaN
222,Тувалу,NaN,NaN,NaN,NaN,NaN
